# AlexNet

In [1]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
#from Stats import WeightQuantization,IntroduceFaultsInWeights,GenerateFaultsList
from Stats_original import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList, CheckAccuracyAndLoss
from Nets import GetNeuralNetworkModel
from Training import GetDatasets
from Simulation import get_all_outputs
from Simulation import buffer_simulation, save_obj, load_obj
from funciones import compilNet, same_elements
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
from openpyxl import Workbook
from openpyxl import load_workbook
import tensorflow_datasets as tfds

In [2]:
## Esta funcion convierte un valor de flotante a su equivalente binario de signo magnitud, asumiendo 11 bits
## para parte fraccionaria (los usados en AlexNet 
def convert(value):
    shift   = 2**(16-1)
    factor  = 2**11
    new_val = value*factor
    new_val = -new_val + shift if new_val < 0 else new_val
    new_val = int(new_val)
    return format(new_val, '#018b')

In [2]:
trainBatchSize = testBatchSize = 1
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)

C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


In [3]:
word_size  = 16
afrac_size = 11  
aint_size  = 4
wfrac_size = 11
wint_size  = 4
# Tamaño del buffer de pesos == al tamaño de la capa con mayor numero de pesos (885120 pesos de 16 bits cada uno)
#wbuffer_size = 885120*word_size
# Tamaño del buffer de activaciones == al tamaño de la capa con mayor numero de activaciones (290400 pesos de 16 bits cada uno)
#abuffer_size = (1024*1024)*word_size
# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'AlexNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')

In [6]:
Cargar_errores = True

#print(load_file('Data/Fault Characterization/wgt/Accs_w_707_55'))

if Cargar_errores:
    locs  = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/locs_054')
    error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/error_mask_054')
locs= locs[0:5]  
error_mask=error_mask[0:5]
    
print('mostrando las 5 primeras direcciones con fallos')
print('direcciones:',locs)
print('mascara de fallos:',error_mask)
print(len(error_mask))
print(len(locs))

mostrando las 5 primeras direcciones con fallos
direcciones: [4141, 4181, 4199, 4241, 4249]
mascara de fallos: ['x0xxxxxxx0xxxxxx', 'x0xxxxxxx0xxxxxx', 'x0xxxxxxx0xxxxxx', 'x0xxxxxxx0xxxxxx', 'x1xxxxxxx0xxxxxx']
5
5


In [4]:
from Simulation import buffer_simulation, save_obj, load_obj
error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/vc_707/error_mask_054')
locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/vc_707/locs_054')
#error_mask=error_mask[0:9]
#locs=locs[0:9]

#locs=locs[100:109]
print(len(error_mask))
print(len(locs))

9439
9439


In [5]:
locs      = [0,2,3]
error_mask = ['xxxx1xxxxxxxxxxx','xxxxxxxxxxxx0xxx','1xxxxxxxxxxxxxxx']

In [7]:
#activation_aging = [True]*11

#activation_aging = [False,False,False,False,True,False,False,False,False,False,False]
#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                             aging_active=True, word_size=word_size, frac_size=afrac_size, 
                             batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
WeightQuantization(model = Net2, frac_bits = 11, int_bits = 4)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
#loss_sf,acc_sf =Net2.evaluate(test_dataset)

AlexNet
index_list [[0, 18, 55, 0],
 [0, 18, 95, 0],
 [0, 18, 113, 0],
 [0, 18, 155, 0],
 [0, 18, 163, 0]]
mod_list [[49087, 0], [49087, 0], [49087, 0], [49087, 0], [65471, 16384]]
dentro de active
dentro de apply round
tensor entero valores afectados  Tensor("lambda_1/SelectV2:0", shape=(5,), dtype=int32)
tensor valores_afectados Tensor("lambda_1/NotEqual:0", shape=(5,), dtype=bool)
valores malos Tensor("lambda_1/GatherNd_1:0", shape=(None,), dtype=int32)
valores buenos tf.print Tensor("lambda_1/GatherNd_2:0", shape=(None,), dtype=int32)
newValues DESPUES de colocar 0 Tensor("lambda_1/SelectV2_17:0", shape=(None,), dtype=int32)
tensor_act que envio para inyectar errores nuevamente Tensor("lambda_1/TensorScatterUpdate:0", shape=(5,), dtype=int32)
new_Values que llega de redondeo Tensor("lambda_1/TensorScatterUpdate:0", shape=(5,), dtype=int32)
tensor luego de aplicar la inyección de errores última Tensor("lambda_1/Cast_1:0", shape=(5,), dtype=float32)
aging_active[0] True
index_list [[

index_list []
mod_list []
AddCustomLayers agin 9 True
AddCustomLayers
aging_active[9] True
index_list []
mod_list []
aging_active[10] True
AddCustomLayers agin 10
Dense output_shape
AddCustomLayers False 1
AddCustomLayers False 2


In [8]:
for i in range(5):
    print(Net2.layers[i])

In [9]:
#iteretor: Toma la primera imagen del dtaset
iterator = next(iter(test_dataset))[0]
#outputs = [layer.output for layer in Net2.layers]  # exclude Input
#layers_fn_inp = K.function([model.input], [model.layers[4].output])
#layers_fn = K.function([Net2.input], [Net2.layers[5].output])
#image = next(iterator)[0]
#out.append(model.layers[5].output)
out=get_all_outputs(Net2,iterator)
out_quantizacion = out[1][0]  ## segunda capa, primer elemento del batch
out_error_and_correction = out[2][0]  ## tercera capa, primer elemento del batch                



dentro de active
dentro de apply round
tensor entero valores afectados  tf.Tensor([1952 1960 1952 1945 1938], shape=(5,), dtype=int32)
tensor_with_error [1952 1960 1952 1945 18322]
tensor valores_afectados tf.Tensor([False False False False  True], shape=(5,), dtype=bool)
valores malos tf.Tensor([18322], shape=(1,), dtype=int32)
valores buenos tf.print tf.Tensor([1938], shape=(1,), dtype=int32)
mask del for [16384]
newValues ANTES  [0]
newValues DESPUES de colocar 0 tf.Tensor([0], shape=(1,), dtype=int32)
tensor_act que envio para inyectar errores nuevamente tf.Tensor([1952 1960 1952 1945    0], shape=(5,), dtype=int32)
new_Values que llega de redondeo tf.Tensor([1952 1960 1952 1945    0], shape=(5,), dtype=int32)
tensor roun luego de inyectar errores entero [1952 1960 1952 1945 16384]
tensor luego de aplicar la inyección de errores última tf.Tensor([0.953125   0.95703125 0.953125   0.94970703 8.        ], shape=(5,), dtype=float32)
affectedValues [0.953125 0.95703125 0.953125 0.949707

tensor valores_afectados tf.Tensor([False False False  True  True], shape=(5,), dtype=bool)
valores malos tf.Tensor([  640 16925], shape=(2,), dtype=int32)
valores buenos tf.print tf.Tensor([704 605], shape=(2,), dtype=int32)
mask del for [64 16384]
newValues ANTES  [767 0]
newValues DESPUES de colocar 0 tf.Tensor([767   0], shape=(2,), dtype=int32)
tensor_act que envio para inyectar errores nuevamente tf.Tensor([33932 33932   569   767     0], shape=(5,), dtype=int32)
new_Values que llega de redondeo tf.Tensor([33932 33932   569   767     0], shape=(5,), dtype=int32)
tensor roun luego de inyectar errores entero [33932 33932 569 703 16384]
tensor luego de aplicar la inyección de errores última tf.Tensor([-0.5683594  -0.5683594   0.27783203  0.34326172  8.        ], shape=(5,), dtype=float32)
affectedValues [-0.568359375 -0.568359375 0.277832031 0.34375 0.295410156]
newValues para actualizar el tensor final [-0.568359375 -0.568359375 0.277832031 0.343261719 8]
dentro de active
dentro de

In [16]:

Ogdtype = valores_afectados.dtype
shift   = 2**(16-1)
factor  = 2**11
tensor  = tf.cast(valores_afectados*factor,dtype=tf.int32)
tensor  = tf.where(tf.less(tensor, 0), -tensor + shift , tensor )
tensor  = tf.bitwise.bitwise_and(tensor,faults[:,0])
tensor  = tf.bitwise.bitwise_or(tensor,faults[:,1])
tensor  = tf.where(tf.greater_equal(tensor,shift), shift-tensor , tensor )
tensor  = tf.cast(tensor/factor,dtype = Ogdtype)
print(tensor)

tf.Tensor([-0.21191406 -0.21191406 -0.21191406 -0.15234375  8.8125    ], shape=(5,), dtype=float32)


In [9]:
Valores_antes   = [out_quantizacion[0][0][0],out_quantizacion[0][2][0],out_quantizacion[0][3][0],out_quantizacion[0][4][0],out_quantizacion[0][5][0]]
Valores_despues = [out_error_and_correction[0][0][0],out_error_and_correction[0][2][0],out_error_and_correction[0][4][0],out_error_and_correction[0][2][0],out_error_and_correction[0][5][0]]

for i in range(5):
    print('Error: ',error_mask[i])
    print('Valor antes: ',Valores_antes[i],'En binario: ',convert(Valores_antes[i]))
    print('Valor con error + aproximacion: ',Valores_despues[i],'En binario: ',convert(Valores_despues[i]))
    print('-------------------------')

Error:  x0xxxxxxx0xxxxxx
Valor antes:  0.953125 En binario:  0b0000011110100000
Valor con error + aproximacion:  0.953125 En binario:  0b0000011110100000
-------------------------
Error:  x0xxxxxxx0xxxxxx
Valor antes:  0.95214844 En binario:  0b0000011110011110
Valor con error + aproximacion:  0.95214844 En binario:  0b0000011110011110
-------------------------
Error:  x0xxxxxxx0xxxxxx
Valor antes:  0.94970703 En binario:  0b0000011110011001
Valor con error + aproximacion:  0.9472656 En binario:  0b0000011110010100
-------------------------
Error:  x0xxxxxxx0xxxxxx
Valor antes:  0.9472656 En binario:  0b0000011110010100
Valor con error + aproximacion:  0.95214844 En binario:  0b0000011110011110
-------------------------
Error:  x1xxxxxxx0xxxxxx
Valor antes:  0.9453125 En binario:  0b0000011110010000
Valor con error + aproximacion:  0.9453125 En binario:  0b0000011110010000
-------------------------


In [2]:
tfds.load('colorectal_histology')


cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'AlexNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')


trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)
#
#
Accs = []
# Accs_w = []
# Accs_a_w = []






#ruta_bin = 'Data/Fault Characterization/VC707/RawData'
#ruta_bin = 'Data/Fault Characterization/KC705_B/RowData/'
#directorio = pathlib.Path(ruta_bin)

#ficheros = [fichero.name for fichero in directorio.iterdir()]
#ficheros.sort()

# for i, j in enumerate(ficheros):
#     directorio= os.path.join(ruta_bin, j)
#     buffer= (analize_file(directorio, buffer_size))
#     error_mask, locs = (buffer_vectores(buffer))
#     print(directorio)
locs = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/locs_054')
error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/error_mask_054')
# error_mask=load_obj('Data/Fault Characterization/error_mask_10/vc_707/error_mask_054')
# locs  = load_obj('Data/Fault Characterization/error_mask_10/vc_707/locs_054')
# error_mask=load_obj('Data/Fault Characterization/error_mask_0x_10/vc_707/error_mask_054')
# locs  = load_obj('Data/Fault Characterization/error_mask_0x_10/vc_707/locs_054')
print('tamaño de locs',len(locs))
print('tamaño de error_mask',len(error_mask))
#print('error_mask',error_mask)



loss,acc   = CheckAccuracyAndLoss('AlexNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                            batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)




Accs.append(acc)
#print(Accs)
print('AlexNet',acc)




C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


tamaño de locs 4738
tamaño de error_mask 4738
AlexNet
dentro de active
tensor dentro de RoundValues Tensor("lambda_1/SelectV2:0", shape=(8448,), dtype=int32)
tensor_with_error Tensor("lambda_1/BitwiseOr:0", shape=(8448,), dtype=int32)
tensor valores_afectados Tensor("lambda_1/NotEqual:0", shape=(8448,), dtype=bool)
affectedValues Tensor("lambda_1/GatherNd:0", shape=(8448,), dtype=float32)
newValues Tensor("lambda_1/Cast_1:0", shape=(8448,), dtype=float32)
newValues Tensor("lambda_1/Cast_1:0", shape=(8448,), dtype=float32)
 final Tensor("lambda_1/TensorScatterUpdate_1:0", shape=(None, 227, 227, 3), dtype=float32)
aging_active[0] True
dentro de active
tensor dentro de RoundValues Tensor("lambda_4/SelectV2:0", shape=(17552,), dtype=int32)
tensor_with_error Tensor("lambda_4/BitwiseOr:0", shape=(17552,), dtype=int32)
tensor valores_afectados Tensor("lambda_4/NotEqual:0", shape=(17552,), dtype=bool)
affectedValues Tensor("lambda_4/GatherNd:0", shape=(17552,), dtype=float32)
newValues Tensor(

UnknownError: Failed to create a NewWriteableFile: Data : Acceso denegado.
; Input/output error [Op:WriteFile]

In [ ]:
X = [x for x,y in test_dataset]
#salidas del modelo sin fallas para la primer imagen del dataset de prueba
outputs1 = get_all_outputs(Net1,X[0])
#salidas del modelo con fallas para la primer imagen del dataset de prueba
outputs2 = get_all_outputs(Net2,X[0])

### Definiciones

In [2]:
# Numero de bits para activaciones (a) y pesos (w)
word_size  = 16
afrac_size = 11  
aint_size  = 4
wfrac_size = 11
wint_size  = 4
# Tamaño del buffer de pesos == al tamaño de la capa con mayor numero de pesos (885120 pesos de 16 bits cada uno)
#wbuffer_size = 885120*word_size
# Tamaño del buffer de activaciones == al tamaño de la capa con mayor numero de activaciones (290400 pesos de 16 bits cada uno)
#abuffer_size = (1024*1024)*word_size
# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'AlexNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')

   ### Generación de una muestra de fallos

In [3]:
# Decidir si cargar errores de un archivo locs y error_mask o generarlos aleatoriamente
Cargar_errores = True

#print(load_file('Data/Fault Characterization/wgt/Accs_w_707_55'))

if Cargar_errores:
    locs  = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/locs_054')
    error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/error_mask_054')
    #locs  = load_obj('Data/Fault Characterization/error_mask y locs_buffer_pesos_vc-707/locs_0_54_buffer_pesos')
    #error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_pesos_vc-707/error_mask_0_54_buffer_pesos')
else:
    
    
    locs = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/locs_0_54')
    numero_bits_con_fallo = len(locs)
    bits_con_fallo = np.random.randint(0,2,numero_bits_con_fallo)
    mbuffer = np.array(['0']*(abuffer_size-numero_bits_con_fallo))
    mbuffer = np.concatenate([mbuffer,bits_con_fallo])
    #Convertirlo en vectores de 16 elementos
    mbuffer = np.reshape(mbuffer,(-1,word_size))
    mbuffer = ["".join(i) for i in mbuffer]
    error_mask=mbuffer[0:numero_bits_con_fallo]
    #more_posiction_errors=len(error_mask)-len(locs)

#print(more_posiction_errors)
print(len(locs)) 
print('mostrando las 5 primeras direcciones con fallos')
print('direcciones:',locs[0:100])
print('mascara de fallos:',error_mask[0:100])
print(len(error_mask))
#print(more_posiction_errors)


4738
mostrando las 5 primeras direcciones con fallos
direcciones: [4141, 4181, 4199, 4241, 4249, 4309, 4410, 4425, 4473, 4525, 4527, 4593, 4661, 4685, 4737, 4789, 4899, 4903, 5001, 5009, 5107, 6836, 7830, 7900, 14395, 14481, 14523, 17254, 18015, 18039, 18073, 18193, 18199, 18343, 18355, 18365, 19243, 23681, 23719, 23863, 28631, 28702, 28788, 28894, 28956, 28958, 29020, 29022, 29150, 29152, 30522, 30526, 30534, 30576, 30586, 30995, 32461, 32683, 34226, 34352, 34400, 34436, 34476, 34602, 34604, 34666, 34716, 38517, 38545, 38583, 38833, 39903, 41111, 41291, 41299, 41449, 41805, 41983, 42869, 43076, 43580, 43658, 43810, 43854, 50849, 50899, 54967, 55913, 55995, 57613, 57679, 57913, 59338, 60952, 64022, 64062, 64280, 64342, 64360, 64428]
mascara de fallos: ['x0xxxxxxx0xxxxxx', 'x0xxxxxxx0xxxxxx', 'x0xxxxxxx0xxxxxx', 'x0xxxxxxx0xxxxxx', 'x1xxxxxxx0xxxxxx', 'x0xxxxxxx1xxxxxx', 'xxxxxx1xxxxxxx1x', 'x1xxxxxxx1xxxxxx', 'x1xxxxxxx1xxxxxx', 'x0xxxxxxx0xxxxxx', 'x1xxxxxxx1xxxxxx', 'x0xxxxxxx1xxxxxx

In [20]:
shape=(227,227,3)
address=64022
actMap = address//(shape[0]*shape[1])
actMapd = address/(shape[0]*shape[1])
print(actMapd)
print(actMap)


1.242446001280832
1


## Cargar el dataset

In [4]:
trainBatchSize = testBatchSize = 1
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)

In [4]:
from Simulation import buffer_simulation, save_obj, load_obj
error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/vc_707/error_mask_054')
locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/vc_707/locs_054')
#error_mask=error_mask[0:9]
#locs=locs[0:9]

#locs=locs[100:109]
print(len(error_mask))
print(len(locs))

9439
9439


### Crear la red sin fallos

In [6]:


activation_aging = [False]*11


#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net1 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, aging_active=activation_aging,
                             word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)
Net1.load_weights(wgt_dir).expect_partial()
#Net1_procces = Net1.get_weights()
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
#WeightQuantization(model = Net1, frac_bits = wfrac_size, int_bits = wint_size)
Net1.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
#Net1_procces = Net1.get_weights()
#loss_sf,acc_sf =Net1.evaluate(test_dataset)

Redondeo


In [6]:

activation_aging = [True]*11

#activation_aging = [False,False,False,False,True,False,False,False,False,False,False]
#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                             aging_active=activation_aging, word_size=word_size, frac_size=afrac_size, 
                             batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
#WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
#loss_sf,acc_sf =Net2.evaluate(test_dataset)

Redondeo
750/750 [==============================] - 74s 98ms/step - loss: 2.6147 - accuracy: 0.4493


# Crear red con fallos en todas las capas a la vez

In [19]:
neighbor_exp=True
activation_aging = [True]*11

#activation_aging = [False,False,False,False,True,False,False,False,False,False,False]
#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                             aging_active=activation_aging, word_size=word_size, frac_size=afrac_size, 
                             batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
#WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
#loss_sf,acc_sf =Net2.evaluate(test_dataset)

Redondeo


## Función para guardar en dataframe to excel

In [20]:

def same_elements(outputs1,outputs2,ciclo,acc_list):
    
    
    write_layer=[2,8,10,16,18,24,30,36,38,44,49,53]
    #FZNET
    #write_layer = [2,6,10,14,18,22,26,30,32,36,39,43]


             
             
    
    for index in range(0,len(outputs2)):
        if index == write_layer[ciclo] and Net2.layers[index].__class__.__name__=='Lambda':
            #for i, layer in enumerate(write_layer):
            print('Capa',index,Net2.layers[index].__class__.__name__)
            a=outputs1[index]== outputs2[index]
            size_output=a.size
            output_true=np.sum(a)
            numero.append(index)
            print('numero', numero)
            capa.append(Net2.layers[index].__class__.__name__)
            print('capa', capa)
            #list_output_true.append(output_true)
            list_size_output.append(size_output)
            amount_dif=size_output-output_true
            print(amount_dif)
            list_amount_dif.append(amount_dif)
            ratio=((amount_dif*100)/size_output)
            list_ratio.append(ratio)
            non_zero = (np.count_nonzero(outputs2[index]))
            total_zero= size_output-non_zero
            ratio_zero=((total_zero*100)/size_output)
            list_ratio_zero.append(ratio_zero)
            list_total_zero.append(total_zero)
    return(numero,capa,list_size_output,list_amount_dif,list_ratio,list_total_zero,list_ratio_zero)        
    #df_numero=pd.DataFrame(numero)
    #df_capa=pd.DataFrame(capa)
    #print('df_capa', df_capa)
    #df_acc=pd.DataFrame(acc_list)
    #df_list_size_output=pd.DataFrame(list_size_output)
    #df_list_output_diff=pd.DataFrame(list_amount_dif)
    #df_list_ratio=pd.DataFrame(list_ratio)
    #df_list_elem_zero=pd.DataFrame(list_total_zero)
    #df_list_ratio_zero=pd.DataFrame(list_ratio_zero)
    #    
    #buf_same_elemen = pd.concat([df_numero,df_capa,df_list_size_output,df_list_output_diff,df_list_ratio,df_acc, df_list_elem_zero, df_list_ratio_zero], axis=1, join='outer')
    #buf_same_elemen.columns = ['Num','Capa','T_actv', 'dif_elem', '%','Acc', 'Act_0', '%']
    #buf_same_elemen.to_excel(writer, sheet_name='ratio_'+ str(architecture), index=False)
                
        
  

## Función me guarda el contenido d etodas las capas en cada iteración en pestañas diferentes seggún la condición del ciclo

In [17]:
def same(outputs1,outputs2,ciclo):
    
    list_size_output=[]
    list_output_true=[]
    list_amount_dif=[]
    list_ratio=[]
    list_ratio_zero=[]
    list_total_zero=[]
    numero=[]
    capa=[]
    


    
    for index in range(0,len(outputs2)):
        
        #print('Capa',index,Net2.layers[index].__class__.__name__)
        a=outputs1[index]== outputs2[index]
        size_output=a.size
        print('size_output', size_output)
        output_true=np.sum(a)
        print('output_true',output_true)
        numero.append(index)
        capa.append(Net2.layers[index].__class__.__name__)
        list_output_true.append(output_true)
        list_size_output.append(size_output)
        print(list_size_output)
        amount_dif=size_output-output_true
        list_amount_dif.append(amount_dif)
        ratio=((amount_dif*100)/size_output)
        list_ratio.append(ratio)
        non_zero = (np.count_nonzero(outputs2[index]))
        total_zero= size_output-non_zero
        ratio_zero=((total_zero*100)/size_output)
        list_ratio_zero.append(ratio_zero)
        list_total_zero.append(total_zero)

        df_numero=pd.DataFrame(numero)
        df_capa=pd.DataFrame(capa)
        df_list_size_output=pd.DataFrame(list_size_output)
        df_list_output_diff=pd.DataFrame(list_amount_dif)
        df_list_ratio=pd.DataFrame(list_ratio)
        df_list_elem_zero=pd.DataFrame(list_total_zero)
        df_list_ratio_zero=pd.DataFrame(list_ratio_zero)
        
        buf_same_elemen = pd.concat([df_numero,df_capa,df_list_size_output,df_list_output_diff, df_list_ratio, df_list_elem_zero, df_list_ratio_zero], axis=1, join='outer')
        buf_same_elemen.columns = ['num','capa','Total_elements_layer', 'diff_elements', 'Ratio', 'amount_zero', 'Ratio']
        buf_same_elemen.to_excel(writer, sheet_name='ratio_'+ str(ciclo), index=False)
  


In [18]:




trainBatchSize = testBatchSize = 1
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)


neighbor_exp=True
activation_aging = np.array([False]*11)
acc_list=[]
list_ciclo=[]
list_size_output=[]
list_size_output=[]
list_output_true=[]
list_amount_dif=[]
list_ratio=[]
list_ratio_zero=[]
list_total_zero=[]
numero=[]
capa=[]

with pd.ExcelWriter('AlexNet/ratio_pruebas_error_mask_borrar.xlsx') as writer:
#from openpyxl import Workbook
#from openpyxl import load_workbook
#wb=Workbook()
#wb.save('AlexNet/ratio_element_diff_pruebas.xlsx')


    for i, valor in enumerate(activation_aging):
        ciclo=i
        print('ciclo',ciclo)
        activation_aging[i]=True 
        activation_aging[i-1]=False    
        print (activation_aging)
    #activation_aging = [False,False,False,False,True,False,False,False,False,False,False]
    #activation_aging= False
        Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=activation_aging, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)
        Net2.load_weights(wgt_dir).expect_partial()
        loss = tf.keras.losses.CategoricalCrossentropy()
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
        Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
        #loss,acc  = Net2.evaluate(test_dataset)
        #acc_list.append(acc)
        #list_ciclo.append(i)
        
        X = [x for x,y in test_dataset]
        #salidas del modelo sin fallas para la primer imagen del dataset de prueba
        outputs1= get_all_outputs(Net1,X[0])
        #salidas del modelo con fallas para la primer imagen del dataset de prueba
        outputs2 = get_all_outputs(Net2,X[0])
        #print(outputs1)
        #print(outputs2)
        #print('list_ciclo',list_ciclo)
        #same_elements(outputs1,outputs2,list_ciclo,acc_list)
        same_elements(outputs1,outputs2,ciclo,acc_list)
## De esta forma es cuando quiero guardar solo las capas que escriben en el buffer
    if ciclo==len(activation_aging)-1:
        last_layer=ciclo+1
        same_elements(outputs1, outputs2, last_layer, 0)
    df_numero=pd.DataFrame(numero)
    df_capa=pd.DataFrame(capa)
    df_acc=pd.DataFrame(acc_list)
    df_list_size_output=pd.DataFrame(list_size_output)
    df_list_output_diff=pd.DataFrame(list_amount_dif)
    df_list_ratio=pd.DataFrame(list_ratio)
    df_list_elem_zero=pd.DataFrame(list_total_zero)
    df_list_ratio_zero=pd.DataFrame(list_ratio_zero)
        
    buf_same_elemen = pd.concat([df_numero,df_capa,df_list_size_output,df_list_output_diff,df_list_ratio,df_acc, df_list_elem_zero, df_list_ratio_zero], axis=1, join='outer')
    buf_same_elemen.columns = ['Num','Capa','T_actv', 'dif_elem', '%','Acc', 'Act_0', '%']
    #buf_same_elemen.to_excel(writer, sheet_name='ratio', index=False)
    #buf_same_elemen.to_excel(writer, sheet_name='ratio', index=False)
    #writer=pd.ExcelWriter('AlexNet/ratio.xlsx', engine='xlsxwriter')

    buf_same_elemen.to_excel(writer,sheet_name='datos1',startcol = 2, index=False)
    writer.save()
    
    #buf_same_elemen.to_excel(writer, writer.worksheets[1].title, startcol = 3,  index = False)
    
    
    
        

        
    
        
print('Ejecución  completada: ', datetime.now().strftime("%H:%M:%S"))   
        
        
    
    


# In[12]:


acc_list


# In[14]:




C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


ciclo 0
[ True False False False False False False False False False False]
Redondeo


AttributeError: 'list' object has no attribute 'layers'

## prueba lueho borro el de arriba si sale bien

In [22]:
#Cargar_errores = True
#
#
#if Cargar_errores:
#    locs  = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/locs_0_54')
#    error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/error_mask_0_54')
#else:
#    numero_bits_con_fallo = 1000
#    bits_con_fallo = np.random.randint(0,2,numero_bits_con_fallo)
#    #crear una mascara (m) del buffer de pesos donde x: bit sin fallo, 0: bit con fallo en 0, 1: bit con fallo en 1.
#    #si quieres introducir fallos en activaciones en lugar de los pesos, simplemente cambia wbuffer_size por abuffer_size.
#    mbuffer = np.array(['x']*(abuffer_size-numero_bits_con_fallo))
#    mbuffer = np.concatenate([mbuffer,bits_con_fallo])
#    #distribuir los errores aleatoriamente en la mascara del buffer
#    np.random.shuffle(mbuffer)
#    #organizar la mascara del buffer por direcciones
#    mbuffer = np.reshape(mbuffer,(-1,word_size))
#    mbuffer = ["".join(i) for i in mbuffer]
#    #filtrar dejando solo las direcciones con error
#    locs  = [x for x,y in enumerate(mbuffer) if y.count('x') < 16]
#    masks = [y for x,y in enumerate(mbuffer) if y.count('x') < 16] 
#print('mostrando las 5 primeras direcciones con fallos')
#print('direcciones:',locs[0:5])
#print('mascara de fallos:',error_mask[0:5])
#print(len(error_mask))



trainBatchSize = testBatchSize = 1
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)


neighbor_exp=True
activation_aging = np.array([False]*11)
acc_list=[]
list_ciclo=[]
list_size_output=[]
list_size_output=[]
list_output_true=[]
list_amount_dif=[]
list_ratio=[]
list_ratio_zero=[]
list_total_zero=[]
numero=[]
capa=[]

with pd.ExcelWriter('AlexNet/ratio_pruebas_error_mask_borrar.xlsx') as writer:
#from openpyxl import Workbook
#from openpyxl import load_workbook
#wb=Workbook()
#wb.save('AlexNet/ratio_element_diff_pruebas.xlsx')


    for i, valor in enumerate(activation_aging):
        ciclo=i
        print('ciclo',ciclo)
        activation_aging[i]=True 
        activation_aging[i-1]=False    
        print (activation_aging)
    #activation_aging = [False,False,False,False,True,False,False,False,False,False,False]
    #activation_aging= False
        Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=activation_aging, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)
        Net2.load_weights(wgt_dir).expect_partial()
        loss = tf.keras.losses.CategoricalCrossentropy()
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
        Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
        loss,acc  = Net2.evaluate(test_dataset)
        #acc_list.append(acc)
        #list_ciclo.append(i)
        
        X = [x for x,y in test_dataset]
        #salidas del modelo sin fallas para la primer imagen del dataset de prueba
        outputs1= get_all_outputs(Net1,X[0])
        #salidas del modelo con fallas para la primer imagen del dataset de prueba
        outputs2 = get_all_outputs(Net2,X[0])
        #print(outputs1)
        #print(outputs2)
        #print('list_ciclo',list_ciclo)
        #same_elements(outputs1,outputs2,list_ciclo,acc_list)
        same_elements(outputs1,outputs2,ciclo,acc_list)
## De esta forma es cuando quiero guardar solo las capas que escriben en el buffer
    if ciclo==len(activation_aging)-1:
        last_layer=ciclo+1
        same_elements(outputs1, outputs2, last_layer, 0)
    df_numero=pd.DataFrame(numero)
    df_capa=pd.DataFrame(capa)
    df_acc=pd.DataFrame(acc_list)
    df_list_size_output=pd.DataFrame(list_size_output)
    df_list_output_diff=pd.DataFrame(list_amount_dif)
    df_list_ratio=pd.DataFrame(list_ratio)
    df_list_elem_zero=pd.DataFrame(list_total_zero)
    df_list_ratio_zero=pd.DataFrame(list_ratio_zero)
        
    buf_same_elemen = pd.concat([df_numero,df_capa,df_list_size_output,df_list_output_diff,df_list_ratio,df_acc, df_list_elem_zero, df_list_ratio_zero], axis=1, join='outer')
    buf_same_elemen.columns = ['Num','Capa','T_actv', 'dif_elem', '%','Acc', 'Act_0', '%']
    #buf_same_elemen.to_excel(writer, sheet_name='ratio', index=False)
    #buf_same_elemen.to_excel(writer, sheet_name='ratio', index=False)
    #writer=pd.ExcelWriter('AlexNet/ratio.xlsx', engine='xlsxwriter')

    buf_same_elemen.to_excel(writer,sheet_name='datos1',startcol = 2, index=False)
    writer.save()
    
    #buf_same_elemen.to_excel(writer, writer.worksheets[1].title, startcol = 3,  index = False)
    
    
    
        

        
    
        
print('Ejecución  completada: ', datetime.now().strftime("%H:%M:%S"))   
        
        
    
    


# In[12]:


acc_list


# In[14]:




C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


ciclo 0
[ True False False False False False False False False False False]
Redondeo
750/750 [==============================] - 70s 93ms/step - loss: 0.7251 - accuracy: 0.7320
Capa 2 Lambda
numero [2]
capa ['Lambda']
570
ciclo 1
[False  True False False False False False False False False False]
Redondeo
750/750 [==============================] - 49s 65ms/step - loss: 1.9566 - accuracy: 0.6187
Capa 8 Lambda
numero [2, 8]
capa ['Lambda', 'Lambda']
1193
ciclo 2
[False False  True False False False False False False False False]
Redondeo
750/750 [==============================] - 50s 67ms/step - loss: 0.5324 - accuracy: 0.7453
Capa 10 Lambda
numero [2, 8, 10]
capa ['Lambda', 'Lambda', 'Lambda']
103
ciclo 3
[False False False  True False False False False False False False]
Redondeo
750/750 [==============================] - 50s 66ms/step - loss: 0.3126 - accuracy: 0.8893
Capa 16 Lambda
numero [2, 8, 10, 16]
capa ['Lambda', 'Lambda', 'Lambda', 'Lambda']
790
ciclo 4
[False False False False

ValueError: Length mismatch: Expected axis has 7 elements, new values have 8 elements

In [79]:
from openpyxl import Workbook
wb=Workbook()
wb.save('AlexNet/ratio_element_diff_pruebas.xlsx')

### Crear la red con fallos en activaciones en ciclo de vectores true and false

In [21]:
activation_aging = np.array([False]*11)
acc_list=[]
list_ciclo=[]


with pd.ExcelWriter('AlexNet_sinageing/ratio_element_diff.xlsx') as writer:
    


    for i, valor in enumerate(activation_aging):
        ciclo=i
        activation_aging[i]=True 
        activation_aging[i-1]=False    
        print (activation_aging)
    #activation_aging = [False,False,False,False,True,False,False,False,False,False,False]
    #activation_aging= False
        Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=activation_aging, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)
        Net2.load_weights(wgt_dir).expect_partial()
        loss = tf.keras.losses.CategoricalCrossentropy()
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
        Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
        loss,acc  = Net2.evaluate(test_dataset)
        acc_list.append(acc)
        print(acc_list)
        #list_ciclo.append(i)
        
        X = [x for x,y in test_dataset]
        #salidas del modelo sin fallas para la primer imagen del dataset de prueba
        outputs1= get_all_outputs(Net1,X[0])
        #salidas del modelo con fallas para la primer imagen del dataset de prueba
        outputs2 = get_all_outputs(Net2,X[0])
        #print(outputs1)
        #print(outputs2)
        #print('list_ciclo',list_ciclo)
        #same_elements(outputs1,outputs2,list_ciclo,acc_list)
        same_elements(outputs1,outputs2,ciclo)
writer.close        

        
    
        
print('Ejecución  completada: ', datetime.now().strftime("%H:%M:%S"))   
        
        

OSError: Cannot save file into a non-existent directory: 'AlexNet_sinageing'

In [38]:
import collections
from numpy import array
write_layer=[2,8,10,16,18,24,30,36,38,44,49,53]
for index in range(0,len(outputs2)):
    #if Net2.layers[index].__class__.__name__== 'Lambda' and write_layer.index(i):
    if index in write_layer and Net2.layers[index].__class__.__name__== 'Lambda':
        print('yes')      
        print('Capa',index,Net2.layers[index].__class__.__name__)
        a=(np.count_nonzero(outputs2[index]))
        b=(outputs2[index].size)
        print(b-a)
        c= b-a
        x= (c*100)/b
        print('Cantidad de elementos en 0:', c , 'de un total de:', b , 'ratio:', x, '%')
             
    

yes
Capa 2 Lambda
0
Cantidad de elementos en 0: 0 de un total de: 154587 ratio: 0.0 %
yes
Capa 8 Lambda
9064
Cantidad de elementos en 0: 9064 de un total de: 290400 ratio: 3.121212121212121 %
yes
Capa 10 Lambda
2153
Cantidad de elementos en 0: 2153 de un total de: 69984 ratio: 3.0764174668495654 %
yes
Capa 16 Lambda
21
Cantidad de elementos en 0: 21 de un total de: 186624 ratio: 0.011252572016460906 %
yes
Capa 18 Lambda
2
Cantidad de elementos en 0: 2 de un total de: 43264 ratio: 0.004622781065088758 %
yes
Capa 24 Lambda
3
Cantidad de elementos en 0: 3 de un total de: 64896 ratio: 0.004622781065088758 %
yes
Capa 30 Lambda
4
Cantidad de elementos en 0: 4 de un total de: 64896 ratio: 0.00616370808678501 %
yes
Capa 36 Lambda
3
Cantidad de elementos en 0: 3 de un total de: 43264 ratio: 0.006934171597633136 %
yes
Capa 38 Lambda
3
Cantidad de elementos en 0: 3 de un total de: 9216 ratio: 0.032552083333333336 %
yes
Capa 44 Lambda
3778
Cantidad de elementos en 0: 3778 de un total de: 4096 rati

## Analizar Accurancy inyectando errores en la última Capa
### Lo haré con la máscara de error del buffer de pesos para el voltaje 0.54

In [13]:
Cargar_errores = True


if Cargar_errores:
    locs  = load_obj('Data/Fault Characterization/error_mask y locs_buffer_pesos_vc-707/locs_0_54_buffer_pesos')
    error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_pesos_vc-707/error_mask_0_54_buffer_pesos')
print('mostrando las 5 primeras direcciones con fallos')
print('direcciones:',locs[0:20])
print('mascara de fallos:',error_mask[0:20])
print(len(error_mask))
print(len(locs))    

mostrando las 5 primeras direcciones con fallos
direcciones: [343, 583, 601, 631, 653, 671, 809, 841, 843, 875, 899, 913, 937, 941, 953, 969, 1312, 1422, 2079, 2179]
mascara de fallos: ['xxxx0xxxxxxx0xxx', 'x0xxxxxxx0xxxxxx', '00xxxxxx00xxxxxx', 'xxxxxx1xxxxxxx0x', '0xxxxxxx1xxxxxxx', 'xxxxxx1xxxxxxx1x', 'xxxx1xxxxxxx1xxx', 'xxxxxx1xxxxxxx1x', 'x0xxxx0xx1xxxx1x', 'xxxxxx0xxxxxxx1x', 'xxxx1xxxxxxx0xxx', 'xxxxxx1xxxxxxx1x', 'xxxxxxx1xxxxxxx0', 'x1xxxxxxx1xxxxxx', 'xxxxxx1xxxxxxx1x', 'xxxxxx1xxxxxxx1x', 'xxxxxx0xxxxxxx1x', 'xxxxxx0xxxxxxx0x', 'xxxx0xxxxxxx0xxx', 'xxxx0xxxxxxx1xxx']
6265
6265


### inyectando errres en toda las capas este es lo que se optiene accuracy: 0.3613

In [121]:
mask='x1xxxxxxx0xxxxxx'
error_1  = int("".join(mask.replace('x','0')),2)
error_0  = int("".join(mask.replace('x','1')),2)
print(error_1)
print(error_0)

16384
65471


In [57]:
if static_1_error !=0:
    static_1_error=static_1_error-1
if  static_0_Error!=0:
    static_0_Error=static_0_Error-1   
print(static_1_error)
print(static_0_Error)

0
49086


In [ ]:
-1=32769
-72=32840

In [122]:
-(-1)+(2**(16-1))

32769

In [70]:
-(-1)+(2**(16-1))

32769

In [37]:
pow(2,11)+8

2056

In [118]:
import tensorflow as tf
from tensorflow.python.ops import bitwise_ops
dtype_list = [tf.int8, tf.int16, tf.int32, tf.int64,
              tf.uint8, tf.uint16, tf.uint32, tf.uint64]

for dtype in dtype_list:
    lhs = tf.constant([0, 5, 3, 14], dtype=dtype)
    rhs = tf.constant([5, 0, 7, 11], dtype=dtype)
    #exp = tf.constant([0, 0, 3, 10], dtype=tf.float32)

    res = bitwise_ops.bitwise_and(lhs, rhs)
    tf.assert_equal(tf.cast(res, tf.float32), exp) # TRUE


In [119]:
res

<tf.Tensor: shape=(4,), dtype=uint64, numpy=array([ 0,  0,  3, 10], dtype=uint64)>

In [124]:
tensor=tf.placeholder=(38,38,38)
a=[(32639,0),(49087,0),(49151,64)]
faults = np.array(a)
print(tensor)
print(faults)

(38, 38, 38)
[[32639     0]
 [49087     0]
 [49151    64]]


In [125]:

tensor  = tf.bitwise.bitwise_and(tensor,faults[:,0])
print(tensor)

tf.Tensor([38 38 38], shape=(3,), dtype=int32)


In [18]:
index_list_np = np.random.rand(4)

In [24]:
index_list_np=([2,3,4,5])

In [27]:
index_vecino=[x+1 for x in index_list_np[1:]]
print('index_vecino ',index_vecino)

index_vecino  [4, 5, 6]


In [9]:

#tf.disable_v2_behavior()
tensor = tf.constant([[11,12,13,14],[14,15,16,15],[17,18,19,16]])
#tensor.eval(ses=tf.Session())

In [10]:
tensor

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[11, 12, 13, 14],
       [14, 15, 16, 15],
       [17, 18, 19, 16]])>

In [11]:
numpy_array=tensor.numpy()

In [12]:
numpy_array

array([[11, 12, 13, 14],
       [14, 15, 16, 15],
       [17, 18, 19, 16]])

In [20]:
index_vecino=[x[1]+1 for x in numpy_array]
print('index_vecino ',index_vecino)

index_vecino  [13, 16, 19]


In [50]:
for i, x in enumerate(numpy_array):
    index_vecino=[x+1 for x[:1] in numpy_array]
    

ValueError: could not broadcast input array from shape (4,) into shape (1,)

In [53]:
print(type(index_vecino))

<class 'list'>
